In [1]:
import numpy as np
import math

In [2]:
class Point :
    def __init__ ( self,number, x, y ):
        self._x = x
        self._y = y
        self._number = number
    @property 
    def number (self):
        return self._number
    @property 
    def x (self):
        return self._x
    @property 
    def y (self):
        return self._y
    def __repr__ (self):
        return f'Point (Number = {self._number}, x={self._x}, y={self._y})'

In [3]:
class aluminium:
    """
    Creates a  AS35016 material object 
    """
    def __init__(self):
        self._e = 10**6 #Pa
        self._ro = 2.6*10**-4 #kg/m^3

    @property
    def e (self):
        return self._e
    
    @property
    def ro (self):
        return self._ro

    def __repr__ (self):
        return f'Aluminium : e:{self._e}ro:{self._ro}'

In [4]:
class EbElement:

    def __init__ (self, nodes, number, material, ii, a):
        #Create a list of layers
        self._nodes = nodes
        self._length = self.length_calculator()
        self._number = number
        self._material = material
        self._i = ii
        self._a = a

    def length_calculator(self):
        node_1 = self._nodes[0]
        node_2 = self._nodes[1]
        return math.sqrt( ( node_2.x - node_1.x )**2 + ( node_2.y - node_1.y )**2 )
    
    @property 
    def number (self):
        return self._number
    @property
    def node1 (self):
        return self._nodes[0]
    
    @property
    def node2 (self):
        return self._nodes[1]
    
    @property    
    def i (self):
        return self._i
    
    @property    
    def a (self):
        return self._a

    @property 
    def length(self):
        return self._length 
    
    @property
    def material (self):
        return self._material
    
    def element_stiffness(self,omega):
        om = omega
        self._element_stiffness_matrix = np.zeros( (4, 4) )
        k1 = ( self.material.ro  * self.a * (om**2) / ( self.material.e * self.i ) ) ** 0.25   
        kl = k1 * self.length
        det = 1 - math.cos ( kl ) * math.cosh (kl)
        stf = self.material.e * self.i / ( self.length ** 3 )
        alfa =  ( math.cos( kl ) * math.sinh( kl ) + math.sin( kl ) * math.cosh( kl ) ) * ( kl ** 3 ) / det
        alfabar = ( math.sin (kl) + math.sinh ( kl ) ) * ( kl ** 3 ) / det    
        beta = ( (-1) * math.cos(kl) * math.sinh(kl) + math.sin(kl) * math.cosh(kl) ) * (kl) / det
        betabar = ( (-1) * math.sin(kl) + math.sinh(kl) ) * (kl) / det
        gama = ( (-1) * math.cos(kl) + math.cosh(kl) ) * ( kl ** 2 ) / det
        gamabar = math.sin(kl) * math.sinh(kl) * (kl**2) / det

        self._element_stiffness_matrix [0,0] = stf * alfa
        self._element_stiffness_matrix [0,1] = stf * gamabar * self.length
        self._element_stiffness_matrix [0,2] = stf * (-1) * alfabar
        self._element_stiffness_matrix [0,3] = stf * gama * self.length
        self._element_stiffness_matrix [1,1] = stf * beta * ( self.length ** 2 )
        self._element_stiffness_matrix [1,2] = stf * (-1) * gama * self.length #!!!!!! It was written gammabar in doyles paper which is wrong
        self._element_stiffness_matrix [1,3] = stf * betabar * ( self.length ** 2 )
        self._element_stiffness_matrix [2,2] = stf * alfa
        self._element_stiffness_matrix [2,3] = stf * (-1) * gamabar * self.length
        self._element_stiffness_matrix [3,3] = stf * beta * (self.length ** 2 ) 
        
        self._element_stiffness_matrix [1,0] = self._element_stiffness_matrix[0,1]
        self._element_stiffness_matrix [2,0] = self._element_stiffness_matrix[0,2]
        self._element_stiffness_matrix [2,1] = self._element_stiffness_matrix[1,2]
        self._element_stiffness_matrix [3,0] = self._element_stiffness_matrix[0,3]
        self._element_stiffness_matrix [3,1] = self._element_stiffness_matrix[1,3]
        self._element_stiffness_matrix [3,2] = self._element_stiffness_matrix[2,3]
        return self._element_stiffness_matrix

    def __repr__ (self):
        return f'Element Number {self.number}'
    

In [5]:
class Structure :
    """
    Structure is an iterable with an inside iterable. 
    You can loop through different elements inside the structure and access 
    physical properties of each element.
    """
    def __init__ (self, elements, fixed):
        #Create a list of layers
        self._elements = elements
        self._nj = len ( self._elements ) + 1
        self._fixed = fixed
    
    def stiffness_global (self, omega):
        self.assemb (omega)
        self.boundary_conditions ()
        return self.stiffness_global_matrix
    
    def assemb (self, omega):
        for ielem, element in enumerate (self):
            if ielem == 0:
                self.stiffness_global_matrix  = np.zeros ( (self.nj*2,self.nj*2), dtype=complex )
            ndofn = 2
            nnode = 2 
            for inode in range (1,nnode+1):
                if inode == 1 :
                    nodei = element.node1.number
                if inode == 2 :
                    nodei = element.node2.number
                for idofn in range (1,ndofn+1):
                    nrows = ( nodei - 1 ) * ndofn + idofn - 1
                    nrowe = ( inode - 1 ) * ndofn + idofn - 1	
                    for jnode in range (1,nnode+1):
                        if jnode == 1 :
                            nodej = element.node1.number
                        if jnode == 2 :
                            nodej = element.node2.number
                        for jdofn in range (1,ndofn+1):
                            ncols = ( nodej - 1 ) * ndofn + jdofn - 1
                            ncole = ( jnode - 1 ) * ndofn + jdofn - 1 
                            self.stiffness_global_matrix [nrows,ncols]= self.stiffness_global_matrix [nrows,ncols] +  element.element_stiffness ( omega )[nrowe,ncole]
        return self.stiffness_global_matrix   
        
    def boundary_conditions (self):
        self.stiffness_global_matrix = np.delete (np.delete (self.stiffness_global_matrix, self._fixed, 0), self._fixed, 1)
        return self.stiffness_global_matrix
        
    @property
    def nj (self):
        return self._nj
    
    @property 
    def fixed (self):
        return self._fixed
    
    def __repr__ (self):
        return f'Structure contains Elements: {[element.number for element in self]}'
    def __len__ (self):
        return len(self._elements)
    
    def __getitem__ (self, s):
        return self._elements[s]
    
    def __iter__ (self):
        return self.ElementIterator(self)
    
    class ElementIterator:
        def __init__ (self, structure_obj) :
            self._structure = structure_obj
            self._index = 0

        def __iter__(self):
            return self

        def __next__(self):
            if self._index >= len(self._structure):
                raise StopIteration
            else:
                item = self._structure._elements[self._index]
                self._index += 1
                return item


In [6]:
class W_w :
    def __init__ (self, structure_obj):
        self._structure = structure_obj
    
    def search ( self, n ) :
        num_freq = n
        eigenvalues_beam = np.empty((num_freq))
        om_u = 0
        step = 1000
        eps = 0.01
        num_iteration = 0
        for i in range (1,num_freq+1):
            k=0
            om = om_u
            num_iteration = 0
            while  num_iteration == 0 :
                k=k+1
                om_l = om
                om_u = om_u + step
                error=1
                j=0
                while error > eps or  j > i  :
                    if (num_iteration==0) :
                        om =om_u
                    else:
                         om = 0.5 * ( om_u + om_l )
                    k = self._structure.stiffness_global (om) 
                    j0 = self.j0_analytical (om)
                    sign = self.upper_triangular (om)
                    j = j0 + sign
                    if j >= i  :
                        om_u = om
                        num_iteration = num_iteration + 1
                    if  j < i :
                        om_l = om
                    error = om_u - om_l
            eigenvalues_beam [ i - 1 ] = om
            om_l = om_u
        return eigenvalues_beam
        
    def upper_triangular ( self, om ) :
        #make upper triangular and count
        sign = 0
        kk = self._structure.stiffness_global (om)
        matrix = np.copy (kk)
        nf = np.shape (kk)[0]
        for k in range ( 0 , nf ):
            for j in range ( k+1 , nf ) :
                c = matrix[j][k] / matrix[k][k]
                for p in range ( 0 , nf ):
                    matrix [j][p] = matrix[j][p] - c * matrix [k][p]
        for i in range (0,nf):
            if matrix[i][i] < 0:
                sign = sign + 1
        return sign
    
    def j0_analytical (self,om):
        j0 = 0
        for ielem,element in enumerate (structure) :
            beam_wavenumber = element.length * ( element.material.ro * element.a * (om**2) / ( element.material.e * element.i ) ) ** 0.25 
            i = int ( beam_wavenumber / np.pi )
            r = 1 - math.cos (beam_wavenumber) * math.cosh (beam_wavenumber)
            if r >= 0.0 :
                sign = 1
            else:
                sign = -1
            jb = i - ( 1 - sign * (-1) ** i  ) / 2
            j0 += jb
        return j0
    
    def j0 ( self, omega) : 
        for element in structure:
            newPoint1 = Point (number = 1, x = element.node1.x, y = element.node1.y)
            newPoint2 = Point (number = 2, x = 0.5 * ( element.node1.x + element.node2.x ) , y = element.node1.y)
            newPoint3 = Point (number = 3, x = element.node2.x, y = element.node1.y)
            newElement_1 = EbElement ( (newPoint1,newPoint2), number=1, material=element.material, ii=element.i, a=element.a)
            newElement_2 = EbElement ( (newPoint2,newPoint3), number=2, material=element.material, ii=element.i, a=element.a)
            newStructure = Structure ( (newElement_1,newElement_2), fixed = [0,1,4,5] )
            print (newElement_2.node1)
            k = newStructure.stiffness_global (omega)
            j = self.upper_triangular (k)
            return j

2 Elements

In [36]:
material_1 = aluminium()
point_1 = Point (number = 1, x = 0, y = 0)
point_2 = Point (number = 2, x = 30, y = 0)
point_3 = Point (number = 3, x = 60, y = 0)
element_1 = EbElement((point_1,point_2) , number=1, material=material_1, ii= 5.33, a=4)
element_2 = EbElement((point_2,point_3) , number=2, material=material_1, ii= 5.33, a=4)

In [37]:
structure = Structure (elements = (element_1,element_2), fixed = [0,4])

In [38]:
my_ww = W_w (structure)

In [39]:
previous = 0
for omega in range (1,10000,1):
    now = my_ww.upper_triangular(omega)
    j0 = my_ww.j0_analytical(omega)
    if now != previous :
        print (f'omega is {omega}, f is {omega/(2*np.pi)}Hz, sign count is {now}, and j0 is {j0}')
    previous = now

omega is 197, f is 31.35352378910338Hz, sign count is 1, and j0 is 0.0
omega is 786, f is 125.09578527022974Hz, sign count is 2, and j0 is 0.0
omega is 1767, f is 281.2267844433791Hz, sign count is 3, and j0 is 0.0
omega is 1780, f is 283.2957987035737Hz, sign count is 1, and j0 is 2.0
omega is 3141, f is 499.9056762516433Hz, sign count is 2, and j0 is 2.0
omega is 4906, f is 780.8141508088386Hz, sign count is 0, and j0 is 4.0
omega is 4907, f is 780.9733057519304Hz, sign count is 1, and j0 is 4.0
omega is 7066, f is 1124.5888278873324Hz, sign count is 2, and j0 is 4.0
omega is 9618, f is 1530.7522426578494Hz, sign count is 1, and j0 is 6.0


3 Elements

In [47]:
material_1 = aluminium()
point_1 = Point (number = 1, x = 0, y = 0)
point_2 = Point (number = 2, x = 20, y = 0)
point_3 = Point (number = 3, x = 40, y = 0)
point_4 = Point (number = 4, x = 60, y = 0)
element_1 = EbElement((point_1,point_2) , number=1, material=material_1, ii= 5.33, a=4)
element_2 = EbElement((point_2,point_3) , number=2, material=material_1, ii= 5.33, a=4)
element_3 = EbElement((point_3,point_4) , number=3, material=material_1, ii= 5.33, a=4)

In [48]:
structure = Structure (elements = [element_1,element_2,element_3], fixed = [0,6])

In [49]:
my_ww = W_w (structure)

In [50]:
previous = 0
for omega in range (1,10000,1):
    now = my_ww.upper_triangular(omega)
    j0 = my_ww.j0_analytical(omega)
    if now != previous :
        print (f'omega is {omega}, f is {omega/(2*np.pi)}Hz, sign count is {now}, and j0 is {j0}')
    previous = now

omega is 197, f is 31.35352378910338Hz, sign count is 1, and j0 is 0.0
omega is 786, f is 125.09578527022974Hz, sign count is 2, and j0 is 0.0
omega is 1767, f is 281.2267844433791Hz, sign count is 3, and j0 is 0.0
omega is 3141, f is 499.9056762516433Hz, sign count is 4, and j0 is 0.0
omega is 4005, f is 637.4155470830408Hz, sign count is 1, and j0 is 3.0
omega is 4907, f is 780.9733057519304Hz, sign count is 2, and j0 is 3.0
omega is 7066, f is 1124.5888278873324Hz, sign count is 3, and j0 is 3.0
omega is 9618, f is 1530.7522426578494Hz, sign count is 4, and j0 is 3.0


4 Elements

In [55]:
material_1 = aluminium()
point_1 = Point (number = 1, x = 0, y = 0)
point_2 = Point (number = 2, x = 15, y = 0)
point_3 = Point (number = 3, x = 30, y = 0)
point_4 = Point (number = 4, x = 45, y = 0)
point_5 = Point (number = 5, x = 60, y = 0)
element_1 = EbElement((point_1,point_2) , number=1, material=material_1, ii= 5.33, a=4)
element_2 = EbElement((point_2,point_3) , number=2, material=material_1, ii= 5.33, a=4)
element_3 = EbElement((point_3,point_4) , number=3, material=material_1, ii= 5.33, a=4)
element_4 = EbElement((point_4,point_5) , number=4, material=material_1, ii= 5.33, a=4)

In [56]:
structure = Structure (elements = [element_1,element_2,element_3,element_4], fixed = [0,8])

In [57]:
my_ww = W_w (structure)

In [58]:
previous = 0
for omega in range (1,10000,1):
    now = my_ww.upper_triangular(omega)
    j0 = my_ww.j0_analytical(omega)
    if now != previous :
        print (f'omega is {omega}, f is {omega/(2*np.pi)}Hz, sign count is {now}, and j0 is {j0}')
    previous = now

omega is 197, f is 31.35352378910338Hz, sign count is 1, and j0 is 0.0
omega is 786, f is 125.09578527022974Hz, sign count is 2, and j0 is 0.0
omega is 1767, f is 281.2267844433791Hz, sign count is 3, and j0 is 0.0
omega is 3141, f is 499.9056762516433Hz, sign count is 4, and j0 is 0.0
omega is 4907, f is 780.9733057519304Hz, sign count is 5, and j0 is 0.0
omega is 7066, f is 1124.5888278873324Hz, sign count is 6, and j0 is 0.0
omega is 7119, f is 1133.0240398712028Hz, sign count is 2, and j0 is 4.0
omega is 9618, f is 1530.7522426578494Hz, sign count is 3, and j0 is 4.0


5 Elements

In [61]:
material_1 = aluminium()
point_1 = Point (number = 1, x = 0, y = 0)
point_2 = Point (number = 2, x = 12, y = 0)
point_3 = Point (number = 3, x = 24, y = 0)
point_4 = Point (number = 4, x = 36, y = 0)
point_5 = Point (number = 5, x = 48, y = 0)
point_6 = Point (number = 6, x = 60, y = 0)
element_1 = EbElement((point_1,point_2) , number=1, material=material_1, ii= 5.33, a=4)
element_2 = EbElement((point_2,point_3) , number=2, material=material_1, ii= 5.33, a=4)
element_3 = EbElement((point_3,point_4) , number=3, material=material_1, ii= 5.33, a=4)
element_4 = EbElement((point_4,point_5) , number=4, material=material_1, ii= 5.33, a=4)
element_5 = EbElement((point_5,point_6) , number=5, material=material_1, ii= 5.33, a=4)

In [62]:
structure = Structure (elements = [element_1,element_2,element_3,element_4,element_5], fixed = [0,10])

In [63]:
my_ww = W_w (structure)

In [64]:
previous = 0
for omega in range (1,10000,1):
    now = my_ww.upper_triangular(omega)
    j0 = my_ww.j0_analytical(omega)
    if now != previous :
        print (f'omega is {omega}, f is {omega/(2*np.pi)}Hz, sign count is {now}, and j0 is {j0}')
    previous = now

omega is 197, f is 31.35352378910338Hz, sign count is 1, and j0 is 0.0
omega is 786, f is 125.09578527022974Hz, sign count is 2, and j0 is 0.0
omega is 1767, f is 281.2267844433791Hz, sign count is 3, and j0 is 0.0
omega is 3141, f is 499.9056762516433Hz, sign count is 4, and j0 is 0.0
omega is 4907, f is 780.9733057519304Hz, sign count is 5, and j0 is 0.0
omega is 7066, f is 1124.5888278873324Hz, sign count is 6, and j0 is 0.0
omega is 9618, f is 1530.7522426578494Hz, sign count is 7, and j0 is 0.0


6 Elements

In [69]:
material_1 = aluminium()
point_1 = Point (number = 1, x = 0, y = 0)
point_2 = Point (number = 2, x = 10, y = 0)
point_3 = Point (number = 3, x = 20, y = 0)
point_4 = Point (number = 4, x = 30, y = 0)
point_5 = Point (number = 5, x = 40, y = 0)
point_6 = Point (number = 6, x = 50, y = 0)
point_7 = Point (number = 7, x = 60, y = 0)
element_1 = EbElement((point_1,point_2) , number=1, material=material_1, ii= 5.33, a=4)
element_2 = EbElement((point_2,point_3) , number=2, material=material_1, ii= 5.33, a=4)
element_3 = EbElement((point_3,point_4) , number=3, material=material_1, ii= 5.33, a=4)
element_4 = EbElement((point_4,point_5) , number=4, material=material_1, ii= 5.33, a=4)
element_5 = EbElement((point_5,point_6) , number=5, material=material_1, ii= 5.33, a=4)
element_6 = EbElement((point_6,point_7) , number=6, material=material_1, ii= 5.33, a=4)

In [70]:
structure = Structure (elements = [element_1,element_2,element_3,element_4,element_5,element_6], fixed = [0,12])

In [71]:
my_ww = W_w (structure)

In [72]:
previous = 0
for omega in range (1,10000,1):
    now = my_ww.upper_triangular(omega)
    j0 = my_ww.j0_analytical(omega)
    if now != previous :
        print (f'omega is {omega}, f is {omega/(2*np.pi)}Hz, sign count is {now}, and j0 is {j0}')
    previous = now

omega is 197, f is 31.35352378910338Hz, sign count is 1, and j0 is 0.0
omega is 786, f is 125.09578527022974Hz, sign count is 2, and j0 is 0.0
omega is 1767, f is 281.2267844433791Hz, sign count is 3, and j0 is 0.0
omega is 3141, f is 499.9056762516433Hz, sign count is 4, and j0 is 0.0
omega is 4907, f is 780.9733057519304Hz, sign count is 5, and j0 is 0.0
omega is 7066, f is 1124.5888278873324Hz, sign count is 6, and j0 is 0.0
omega is 9618, f is 1530.7522426578494Hz, sign count is 7, and j0 is 0.0


It can be seen that j0 makes a difference. When the number of elements is increased, j0 is kept to zero and the generated frequencies match the analytical. In the case with 2 elements only you can see some wrong bending frequencies are produced after the time j0 is more than 0 like 1780,4906